In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

In [2]:
data_train = pd.read_csv("x_train.csv", sep=';')

In [3]:
data_test = pd.read_csv("x_test.csv", sep=';')

In [4]:
ans_train = pd.read_csv("y_train.csv", )

In [49]:
pol = PolynomialFeatures(degree=3)
data = pol.fit_transform(data_train)

In [55]:
pol = PolynomialFeatures(degree=3)
data_test_pol = pol.fit_transform(data_test)

In [52]:
X_train, X_test, y_train, y_test = train_test_split(data, ans_train.as_matrix().flatten(), test_size=0.33, random_state=42)

In [53]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.cross_validation import train_test_split
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe


SEED = 42

def score(params):
    print("Training with params: ")
    print(params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_test, label=y_test)
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    gbm_model = xgb.train(params, dtrain, num_round,
                          evals=watchlist,
                          verbose_eval=False)
    predictions = gbm_model.predict(dvalid,
                                    ntree_limit=gbm_model.best_iteration + 1)
    score = 1 - log_loss(y_test, predictions)
    
    print("\tScore {0}\n\n".format(score))
    return {'loss': score, 'status': STATUS_OK}


def optimize(
             #trials, 
             random_state=SEED):
    """
    This is the optimization function that given a space (space here) of 
    hyperparameters and a scoring function (score here), finds the best hyperparameters.
    """

    space = {
        'n_estimators': hp.quniform('n_estimators', 5, 1000, 1),
        'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
        # A problem with max_depth casted to float instead of int with
        # the hp.quniform method.
        'max_depth':  hp.choice('max_depth', np.arange(1, 20, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'gamma': hp.quniform('gamma', 0.2, 1, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
        'eval_metric': 'logloss',
        'objective': 'binary:logistic',
        'nthread': 8,
        'booster': 'gbtree',
        'tree_method': 'exact',
        'silent': 1,
        'seed': random_state
    }
    # Use the fmin function from Hyperopt to find the best hyperparameters
    best = fmin(score, space, algo=tpe.suggest, 
                # trials=trials, 
                max_evals=25)
    return best

In [54]:
best_hyperparams = optimize(
                            #trials
                            )
print("The best hyperparameters are: ", "\n")
print(best_hyperparams)

Training with params: 
{'n_estimators': 972.0, 'colsample_bytree': 0.75, 'nthread': 8, 'eval_metric': 'logloss', 'min_child_weight': 4.0, 'objective': 'binary:logistic', 'silent': 1, 'seed': 42, 'subsample': 0.9500000000000001, 'eta': 0.17500000000000002, 'tree_method': 'exact', 'max_depth': 16, 'booster': 'gbtree', 'gamma': 1.0}
	Score 0.5072577647876284


Training with params: 
{'n_estimators': 977.0, 'colsample_bytree': 0.6000000000000001, 'nthread': 8, 'eval_metric': 'logloss', 'min_child_weight': 9.0, 'objective': 'binary:logistic', 'silent': 1, 'seed': 42, 'subsample': 0.75, 'eta': 0.47500000000000003, 'tree_method': 'exact', 'max_depth': 9, 'booster': 'gbtree', 'gamma': 0.75}
	Score 0.3568572695816533


Training with params: 
{'n_estimators': 961.0, 'colsample_bytree': 0.9500000000000001, 'nthread': 8, 'eval_metric': 'logloss', 'min_child_weight': 5.0, 'objective': 'binary:logistic', 'silent': 1, 'seed': 42, 'subsample': 0.8, 'eta': 0.1, 'tree_method': 'exact', 'max_depth': 13, 

In [88]:
model = RandomForestRegressor(n_estimators=3000, n_jobs=-1, verbose=1, max_depth=10, criterion='mse')

model.fit(data, ans_train.as_matrix().flatten())

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   45.3s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 3000 out of 3000 | elapsed:  5.2min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=3000, n_jobs=-1, oob_score=False,
           random_state=None, verbose=1, warm_start=False)

{'n_estimators': 662.0, 'eta': 0.4, 'gamma': 0.30000000000000004, 'subsample': 0.5, 'min_child_weight': 8.0, 'max_depth': 18, 'colsample_bytree': 0.65}

In [60]:
xgb_model = xgb.XGBRegressor(n_estimators=662,
                             colsample_bytree=0.65,
                             objective='binary:logistic',
                             seed=42, gamma=0.3, 
                             min_child_weight=8,
                             learning_rate=0.4,
                             max_depth=18, subsample=0.5)

xgb_model.fit(data, ans_train.as_matrix().flatten())

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.65,
       gamma=0.3, learning_rate=0.4, max_delta_step=0, max_depth=18,
       min_child_weight=8, missing=None, n_estimators=662, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=42, silent=True, subsample=0.5)

In [61]:
pred = xgb_model.predict(data_test_pol)

In [64]:
pred

array([  4.63528093e-03,   3.01649012e-02,   9.44289386e-01, ...,
         1.38049378e-04,   6.20892271e-04,   6.37418255e-02], dtype=float32)

In [89]:
pred = model.predict(data_test_pol)

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done 3000 out of 3000 | elapsed:    1.2s finished


In [63]:
pred.tofile("anses", sep='\n')

In [38]:
pred.shape

(8346,)

In [39]:
y_test.shape

(8346, 1)

In [39]:
import os

ans = []

for elem in os.listdir('submissions/'):
    data = pd.read_csv('submissions/' + elem)
    ans.append(data)

In [41]:
average = ans[0] + ans[1]+ ans[2]+ ans[3]+ ans[4]+ ans[5]+ ans[6]+ ans[7]+ ans[8]

In [43]:
average /= 9

In [48]:
average.as_matrix().flatten().tofile("anses", sep='\n')